In [2]:
# -----------------------------
# Python: Generate JSON for frontend
# -----------------------------
import pandas as pd
import json

# Load hybrid SARIMA-LSTM forecast
df = pd.read_csv("hybrid_sarima_lstm_forecasts.csv")
df['month'] = pd.to_datetime(df['forecast_date']).dt.month.astype(str).str.zfill(2)

# Define seasons
season_mapping = {
    'rainy': ['06','07','08','09','10','11'],
    'school': ['06','07','08','09'],
    'halloween': ['10','11'],
    'christmas': ['09','10','11','12'],
    'valentines': ['02'],
    'holyweek': ['03','04'],
    'summer': ['03','04','05'],
    'back-to-school': ['06','07','08','09']
}

# Assign season
def assign_season(month):
    for s, months in season_mapping.items():
        if month in months:
            return s
    return 'other'

df['season'] = df['month'].apply(assign_season)

# Aggregate top products per season
top_products = df.groupby(['season','category'])['predicted_sold_per_month'].sum().reset_index()
top_products = top_products.sort_values(['season','predicted_sold_per_month'], ascending=[True,False])

# Make JSON for frontend
season_dict = {}
for s in top_products['season'].unique():
    season_dict[s] = top_products[top_products['season']==s]['category'].tolist()[:10]  # top 10 products

# Export JSON
with open('season_top_products.json','w') as f:
    json.dump(season_dict, f)

print("JSON exported: season_top_products.json")

JSON exported: season_top_products.json
